In [31]:
import pandas as pd
import os
import glob

In [13]:
#Splits up the dataset into chunks of 500,000 rows each
for i,chunk in enumerate(pd.read_csv('2022_place_canvas_history.csv.gzip', chunksize=500000, compression='gzip')):
   chunk.to_csv('./tmp/split_csv_pandas/chunk{}.csv'.format(i), index=False)

In [33]:
#Counts how many rows is in the dataset and prints it, could be done better...
path = './tmp/split_csv_pandas'
csv_files = glob.glob(os.path.join(path, "*.csv"))

len = 0
for i, df_chunk in enumerate(csv_files):
    len += pd.read_csv(df_chunk).shape[0]
print('This dataset is: ', len, ' rows long')


This dataset is:  160353104  rows long


In [37]:
#Loops throuh the entire dataset and count how many occurences of a specific user_id
path = './tmp/split_csv_pandas'
csv_files = glob.glob(os.path.join(path, "*.csv"))

for i, df_chunk in enumerate(csv_files):
    df_temp = pd.read_csv(df_chunk)
    print('Now Reading chunk ', i)
    freq = pd.DataFrame()
    for j in df_temp:
        freq = df_temp['user_id'].value_counts()
    freq.to_csv('./tmp/user_list_split_csv/chunk{}.csv'.format(i), index=False)

Now Reading chunk  0
Now Reading chunk  1
Now Reading chunk  2
Now Reading chunk  3
Now Reading chunk  4
Now Reading chunk  5
Now Reading chunk  6


KeyboardInterrupt: 

In [ ]:
#Combines every user_list_csv chunk 
user_list = pd.DataFrame()
user_list.to_csv('./results/user_id_count.csv',index=True, sep=',', encoding='utf-8')
 user_list = pd.concat([user_list, freq])

In [ ]:
#Skriver ut chunkene til en csv fil, kinda pointless :/
df = pd.concat((chunk for chunk in pd.read_csv('2022_place_canvas_history.csv',chunksize=1000000)))
df.to_csv('./results/out.csv', sep=',', encoding='utf-8', index=True)